In [ ]:
import pandas as pd
import numpy as np
import duckdb as duck
import pyarrow as pa
import polars as pl
import seaborn as sns
import seaborn.objects as so
from matplotlib import pyplot as plt

con = duck.connect(database='/home/garcia-ln/Documentos/real-state-prices/data/processed/real_state.duckdb')

## JSON > Parquet
Before starting the process of cleaning and transforming the data for  our analysis, we're gonna make sure to convert the files into `.parquet` format so that we're always dealing with optimazed performance datasets, no matter the situation.  

For that, we're gonna start by loading our data into [Pola.rs](pola.rs 'Most Efficient DataFrame Lib for Python') dataframes and try to get some information from our dataset.

In [ ]:
sp_dt = pl.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/sp_properties.json')
rj_dt = pl.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/rj_properties.json')
pa_dt = pl.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/pa_properties.json')
bh_dt = pl.read_json('/home/garcia-ln/Documentos/real-state-prices/data/raw/bh_properties.json')

#sp_dt.write_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/sp_properties.parquet')
#rj_dt.write_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/rj_properties.parquet')
#pa_dt.write_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/pa_properties.parquet')
#bh_dt.write_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/bh_properties.parquet')

In [ ]:
sql_sp = '''
    CREATE TABLE sp_tbl as 
        SELECT * FROM '~/Documentos/real-state-prices/data/processed/sp_properties.parquet';
    ALTER TABLE sp_tbl
        ADD COLUMN city VARCHAR DEFAULT 'Sao_Paulo'
'''

sql_rj = '''
    CREATE TABLE rj_tbl as 
         SELECT * FROM '~/Documentos/real-state-prices/data/processed/rj_properties.parquet';
    ALTER TABLE rj_tbl
        ADD COLUMN city VARCHAR DEFAULT 'Rio_de_Janeiro'
'''

sql_pa = '''
    CREATE TABLE pa_tbl as 
        SELECT * FROM '~/Documentos/real-state-prices/data/processed/pa_properties.parquet';
    ALTER TABLE pa_tbl
        ADD COLUMN city VARCHAR DEFAULT 'Porto_Alegre'
'''

sql_bh = '''
    CREATE TABLE bh_tbl as 
        SELECT * FROM '~/Documentos/real-state-prices/data/processed/bh_properties.parquet';
    ALTER TABLE bh_tbl
        ADD COLUMN city VARCHAR DEFAULT 'Belo_Horizonte'
'''

con.execute(sql_sp).fetchall()
sp_df = con.table('sp_tbl').df()
display(sp_df)


con.execute(sql_rj).fetchall()
rj_df = con.table('rj_tbl').df()
display(rj_df)


con.execute(sql_pa).fetchall()
pa_df = con.table('pa_tbl').df()
display(pa_df)


con.execute(sql_bh).fetchall()
bh_df = con.table('bh_tbl').df()
display(bh_df)

## Dtypes

Now that we altered the file from `.json` to `.parque` and added the feature to our dataset we're gonna **add all the tables together and define the dtypes of our data**.  


After that we're gonna make sure to **change all dtypes of our dataset**, to keep a tidy dataset for our cleaning, analysis and modeling.

In [ ]:
sql = '''
    CREATE TABLE properties as
        SELECT * FROM sp_tbl 
    UNION ALL 
        SELECT * FROM rj_tbl 
    UNION ALL 
        SELECT * FROM pa_tbl 
    UNION ALL 
        SELECT * FROM bh_tbl
'''

#con.execute(sql).fetchall()

properties = pl.from_pandas(con.table('properties').df())
properties

In [ ]:
# The code for change dtypes on Polars
#properties = properties.with_columns(
#    [
#        (pl.col('type').cast(pl.Categorical)),
#        (pl.col('city').cast(pl.Categorical)),
#        (pl.col('address').cast(pl.Categorical)),
#        (pl.col('neighborhood').cast(pl.Categorical)),
#        (pl.col('footage').cast(pl.Int16)),
#        (pl.col('doorms').cast(pl.Int8)),
#        (pl.col('garages').cast(pl.Int8)),
#        (pl.col('price').cast(pl.Int32))
#    ]
#)
#
#properties.write_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/properties.parquet')
properties = pl.read_parquet('/home/garcia-ln/Documentos/real-state-prices/data/processed/properties.parquet')

In [ ]:
## here's the SQL query for changing the dtypes of the properties table

con.execute('''
    ALTER TABLE properties
        ALTER type SET DATA TYPE VARCHAR;
    ALTER TABLE properties
        ALTER city SET DATA TYPE VARCHAR;
    ALTER TABLE properties
        ALTER address SET DATA TYPE VARCHAR;
    ALTER TABLE properties
        ALTER neighborhood SET DATA TYPE VARCHAR;
    ALTER TABLE properties    
        ALTER footage SET DATA TYPE SMALLINT;
    ALTER TABLE properties    
        ALTER doorms SET DATA TYPE INT2;
    ALTER TABLE properties
        ALTER garages SET DATA TYPE INT2;
    ALTER TABLE properties    
        ALTER price SET DATA TYPE INT4
'''
)

## EDA
Now it's time for one of the most important part of a data job: the analysis. Here we're gonna focus on undertanding the data that we have, its 

We're gonna start by understanding the basic information on the qualy and quant features, followed by some visualizations to help on the insights for our analysis.  

Before we go on, lets make some changes on our dataset to make sure we'll be able to work on it. In this case, i'm gonna use `seaborn` for our dataviz (wich requires the DF on `pandas` format, and not `polars`) insted of using `plotly express` (wich we can use the `polars` DF and generates interactive plots). The reason for that, is for prettier dataviz made simple and easy, seaborn is the way to go and given that we don't have such a big df, there's no problem transforming the pl.df to pd.df just for plotting.  

Said that, we're gonna by transforming the properties df.

In [ ]:
display(properties.null_count())

null = properties.filter(pl.col('type')==None).to_pandas()
df = properties.to_pandas()

In [ ]:
properties['type'].value_counts().unique()

From this, we can observe that we have a very small volume of missing values. But before we do anything with it, lets check if we have those null values concentrated on a group, or if it's well distributed through all citys and prices. After that, we decide whether to **drop those null values, or make some statistical interpolation**.

In [ ]:
display(df.describe(), df.info(), null.describe(), null.info())

From this, we can see that we have some dirty data on out df, with records of properties with no doorms, and less than 10m². On some steps ahead, we're gonna clean or data propertly.

In [ ]:
sns.set_theme(
    context='notebook', 
    style='darkgrid', 
    font_scale=1,
)

sns.catplot(
    kind='count',
    data=null, x='city', 
    height=4, aspect=1.3,  
    orient='v', 
    alpha=.75, linewidth=1
).set_axis_labels('City', 'Count').set_xticklabels(['SP', 'RJ', 'PA', 'BH']).set(title='Null Count/City')


In [ ]:
sns.catplot(
    kind='box',
    data=null, x='price', y='city',   
    orient='h',
    height=5, aspect=2
).set_axis_labels('Prices', 'Cities').set_xticklabels(
    labels=['', '$0', '$500K', '$1M', '$1.5M', '$2M',
            '$2.5M', '$3M', '$3.5M', '$4M'], 
    rotation=45
)
#plt.ticklabel_format(axis='x', style='plain')
plt.title('Null Prices/City')
plt.show()

In [ ]:
hist=so.Plot(
        null, 
        x="price", 
        y=None, 
        color="city").add(
                so.Bars(), 
                so.Hist(bins=15, stat='count'), 
                so.Dodge(),
                )

hist.label(title='Null Price/City', x='Price', y='Percentage').layout(size=(10, 5)).scale(
        x = so.Continuous().tick(every=500_000).label(like='${x:,.0f}')
).limit(x=(0, 3_500_000)).show()

This plots can make easy for us to understand that São Paulo has the majority of null type records, that's important for us when we proceed to use some interpolation to fill those empty data.

In [ ]:
sns.set_theme(
    context='notebook', 
    style='darkgrid', 
    font_scale=1.55,
)


sns.catplot(
    kind='boxen',
    data=df, y='price', col='type', x='city', col_wrap=3,
    palette='ch:s=0, r=2, l=.4, d=.75', orient='v',
    height=10, aspect=.75
).set_axis_labels('Cities', 'Prices').set_yticklabels(
    labels=['', '$0', '$5M', '$10M', '$15M', '$20M',
            '$25M', '$30M', '$35M', '$40M', '$45M'], 
    rotation=45
)
#plt.ticklabel_format(axis='y', style='plain')
plt.title('Prices/Type/City')
plt.show()

In [ ]:
sns.displot(data=df, x='price', hue='type', col='city', col_wrap=2, 
    kind='hist', log_scale=True, element='step', 
    palette='ch:s=1, r=2, l=.3, d=.5', height=5, aspect=1
).set_axis_labels('Prices (R$)')

plt.title('Prices/City/Type')
plt.show()

In [ ]:
sns.relplot(
    data=df, x='price', y='footage', kind='scatter',
    hue='type', col='city', col_wrap=2, style='type', size='doorms',
    palette='ch:s=0, r=.5, l=.2, d=.75',
).set_axis_labels('Prices (R$)', 'Footage (m²)').set_xticklabels(
    labels=['', '$0', '$5M', '$10M', '$15M', '$20M',
            '$25M', '$30M', '$35M', '$40M', '$45M'], 
    rotation=45
)

plt.title('Prices/City/Footage')
plt.show()

In [ ]:
sns.heatmap( 
    data=df.corr(
        method='pearson', 
        numeric_only=True
    ),
    linewidths=.5, 
    linecolor='w', 
    annot=True
)
    
plt.show()

As we can observe from the df plots, we have some outliers, specialy on SP and RJ (with prices >$10M, that we have to deal with, in order to have a better analysis and model.  

Latter on, we're gonna see the best way to deal with it. For now, we just gonna create a filter from some values for better visualizations.

In [ ]:
df_2 = properties.filter(
    (
        (pl.col('type') == 'Apartamento') | 
        (pl.col('type') == 'Casa') | 
        (pl.col('type') == 'Cobertura')
    ) & 
    (pl.col('price') < 10_000_000) & 
    (pl.col('footage') >= 10) & 
    (pl.col('footage') <= 1_000) &
    (pl.col('doorms') >= 1),
)

df_2 = df_2.to_pandas()

display(df_2['type'].value_counts(), df_2.describe())

In [ ]:
sns.heatmap( 
    data=df_2.corr(
        method='pearson', 
        numeric_only=True
    ),
    linewidths=.5, 
    linecolor='w', 
    annot=True
)
    
plt.show()

In [ ]:
sns.set_theme(
    context='notebook', 
    style='darkgrid', 
    font_scale=1,
)

sns.relplot(
    data=df_2, x='price', y='footage', kind='scatter',
    col='city', col_wrap=2, size='doorms',
    height=4, aspect=1
).set_axis_labels('Prices (R$)', 'Footage (m²)').set_xticklabels(
    labels=[' ', '$0', '$2M', '$4M', '$6M', '$8M', '$10M', '$12M'], 
    rotation=45
)

plt.title('Prices/City/Footage')
plt.show()

In [ ]:
sns.set_theme(
    context='notebook', 
    style='darkgrid', 
    font_scale=1
)

sns.displot(
    data=df_2, x='price', col='city', col_wrap=2, 
    kind='hist', log_scale=True, element='step', 
    height=5, aspect=1
).set_axis_labels('Prices (R$)')

plt.show()

In [ ]:
sns.set_theme(
    context='notebook', 
    style='darkgrid', 
    font_scale=1
)

sns.catplot(
    kind='boxen',
    data=df_2, y='price', x='city',
    orient='v', height=5, aspect=1.3
).set_axis_labels('Cities', 'Prices').set_yticklabels(
    labels=['', '$0', '$2M', '$4M', '$6M', '$8M', '$10M', '$12M'], 
    rotation=45
)
#plt.ticklabel_format(axis='y', style='plain')
plt.show()